In [ ]:
import transformers
import os
import json
import datasets
import torch
import tqdm
import random
import logging
from transformers import DebertaTokenizer, DebertaForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy
from sklearn.metrics import f1_score,precision_score,recall_score

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dataset = json.load(open("/content/drive/MyDrive/Complete dataset/Complete_dataset/train.json"))
test_dataset = json.load(open("/content/drive/MyDrive/Complete dataset/Complete_dataset/test.json"))
files = os.listdir("/content/drive/MyDrive/Complete dataset/Complete_dataset/CT json/")
files.remove(".DS_Store")

In [ ]:
!pip install transformers datasets
from datasets import Dataset

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
import os
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", num_labels=2, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Define the preprocessing function to convert the input text into the format expected by the model
def preprocess_function(primary_section, secondary_section=None):
    if secondary_section is None:
        return {"input_ids": tokenizer(primary_section, truncation=True, padding="max_length")["input_ids"],
                "attention_mask": tokenizer(primary_section, truncation=True, padding="max_length")["attention_mask"]}
    else:
        return {"input_ids": tokenizer(primary_section, secondary_section, truncation=True, padding="max_length")["input_ids"],
                "attention_mask": tokenizer(primary_section, secondary_section, truncation=True, padding="max_length")["attention_mask"]}

In [ ]:
#Load the training data from the CT.json and train.json files
ct_data_path = "/content/drive/MyDrive/Complete dataset/CT json"
train_data_path = "/content/drive/MyDrive/Complete dataset/train.json"

with open(train_data_path) as json_file:
    train_data = json.load(json_file)

In [ ]:
# Dev set
import json

dev_data_path = "/content/drive/MyDrive/Task-2-SemEval-2024-main/training_data/dev.json"
with open(dev_data_path) as json_file:
    dev = json.load(json_file)

# Example instance
print(dev[list(dev.keys())[1]])

{'Type': 'Comparison', 'Section_id': 'Eligibility', 'Primary_id': 'NCT00425854', 'Secondary_id': 'NCT01224678', 'Statement': 'Patients with significantly elevated ejection fraction are excluded from the primary trial, but can still be eligible for the secondary trial if they are 55 years of age or over', 'Label': 'Contradiction'}


In [ ]:
uuid_list = list(dev.keys())
statements = []
gold_dev_primary_evidence = []
gold_dev_secondary_evidence = []
for i in range(len(uuid_list)):
  #Retrieve all statements from the development set
  statements.append(dev[uuid_list[i]]["Statement"])

In [ ]:
from datasets import DatasetDict
#Process the training data
train_data = {"input_ids": [], "attention_mask": []}
train_dataset = DatasetDict({"train": Dataset.from_dict(train_data)})

for i in range(len(uuid_list)):
    primary_ctr_path = os.path.join(ct_data_path, dev[uuid_list[i]]["Primary_id"]+".json")
    with open(primary_ctr_path) as json_file:
        primary_ctr = json.load(json_file)

    primary_section = primary_ctr[dev[uuid_list[i]]["Section_id"]]

    if dev[uuid_list[i]]["Type"] == "Comparison":
        secondary_ctr_path = os.path.join(ct_data_path, dev[uuid_list[i]]["Secondary_id"]+".json")
        with open(secondary_ctr_path) as json_file:
            secondary_ctr = json.load(json_file)
        secondary_section = secondary_ctr[dev[uuid_list[i]]["Section_id"]]

        input_data = preprocess_function(primary_section, secondary_section)
    else:
        input_data = preprocess_function(primary_section)

   # Add the data to the train_data dictionary
    train_data["input_ids"].append(input_data["input_ids"])
    train_data["attention_mask"].append(input_data["attention_mask"])

# Convert the dictionary to a Dataset object
train_dataset["train"] = Dataset.from_dict(train_data)

ValueError: batch length of `text`: 26 does not match batch length of `text_pair`: 17.

In [ ]:
#Set up the training arguments and create the Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

In [ ]:
#Load the test data from the test.json file
test_data_path = "/content/drive/MyDrive/training_data/test.json"

with open(test_data_path) as json_file:
    test_data = json.load(json_file)

In [ ]:
#Preprocess the test data using the same preprocessing function as before
test_dataset = datasets.Dataset.from_dict({"input_ids": [], "attention_mask": []})
for entry in test_data:
    primary_ctr_path = os.path.join(ct_data_path, entry["Primary_id"] + ".json")
    with open(primary_ctr_path) as json_file:
        primary_ctr = json.load(json_file)

    primary_section = primary_ctr[entry["Section_id"]]

    if entry["Type"] == "Comparison":
        secondary_ctr_path = os.path.join(ct_data_path, entry["Secondary_id"] + ".json")
        with open(secondary_ctr_path) as json_file:
            secondary_ctr = json.load(json_file)
        secondary_section = secondary_ctr[entry["Section_id"]]

        input_data = preprocess_function(primary_section, secondary_section)
    else:
        input_data = preprocess_function(primary_section)

    test_dataset.add_row(input_data)

In [ ]:
#Make predictions using the fine-tuned model:
predictions = trainer.predict(test_dataset).predictions

In [ ]:
results = {}
for i, prediction in enumerate(predictions):
    uuid = uuid_list[i]
    results[uuid] = {"Prediction": "Entailment" if prediction > 0.5 else "Contradiction"}

with open("results.json", "w") as json_file:
    json.dump(results, json_file)

In [ ]:
print(Results)
with open("/content/drive/MyDrive/results.json",'w') as jsonFile:
    jsonFile.write(json.dumps(Results,indent=4))

{'1adc970c-d433-44d0-aa09-d3834986f7a2': {'Prediction': 'Entailment'}, '6b9162d0-0816-46d4-81af-c60028dcc63b': {'Prediction': 'Entailment'}, '0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f': {'Prediction': 'Entailment'}, 'cc1f712a-2116-4e40-9810-f315e3fa5ff8': {'Prediction': 'Entailment'}, '904061c0-14fa-4f13-9118-9a41e24fa8eb': {'Prediction': 'Entailment'}, '43ee7645-ce1e-42d5-9a74-3e379f6f367b': {'Prediction': 'Entailment'}, '0cef8c8e-7986-46c7-a597-c5733a9899c0': {'Prediction': 'Entailment'}, '43ce26e5-03fa-4e9d-b0eb-6ea356295753': {'Prediction': 'Entailment'}, '3facad41-0221-42f8-834d-470e65c4aad5': {'Prediction': 'Entailment'}, '9cbc00e9-3a2d-4471-a93e-72c95132fb6a': {'Prediction': 'Entailment'}, '8b91cab9-d858-45f3-bf8d-3d6fc55b4818': {'Prediction': 'Entailment'}, '4a75574c-fa86-4e62-a210-81c7b98a3807': {'Prediction': 'Entailment'}, 'd0b50aeb-aad8-4a8d-aae6-5c58a7d382c7': {'Prediction': 'Entailment'}, 'b0b61978-57db-4a1c-812c-509e8b05f2dc': {'Prediction': 'Entailment'}, '24b85b44-b9e6-4c28